## Intro

In this Notebook I will train and evaluate two different Trigram models.
More precisely I will use two different approaches for creating a Trigram model.
Approach 1 is to build a matrix that holds the true probability distribution of which character 
comes next given that the model has been given two particular preceeding characters.
We then sample from this distribution to produce the next character in the output 'name'.

The second approach is to use a Neural Net. We take in the two preceeding characters to generate the next one 
and so on. The goal is to achieve the same performance or loss using the Neural Net approach as with approach 1.
As you will see approach 1 is the perfect approach given the loss function I will use. 
It is impossible to achieve better loss with a Neural Net approach to the Trigram model.

## First approach

We begin by loading in all the names from the text file:

In [7]:
print('hello')

hello
